In [1]:
from bs4 import BeautifulSoup
import requests
import re
from tqdm import tqdm
import scholarly
import pandas as pd
import numpy as np
import math

import io
import requests
import PyPDF2
from PyPDF2 import PdfFileReader, utils
from bs4 import BeautifulSoup
import urllib.request as urllib2
from tqdm import tqdm

from io import StringIO
import subprocess

from psutil import process_iter
import time
from time import sleep

import matplotlib.pyplot as plt
import textract
import slate3k as slate

from pdfminer.layout import LAParams
from pdfminer.pdfinterp  import PDFResourceManager
from pdfminer.pdfinterp  import PDFPageInterpreter

from pdfminer.converter import PDFPageAggregator

from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBoxHorizontal
from urllib.parse import urljoin

In [2]:
def get_text(url):
    r = requests.get(url)
    f = io.BytesIO(r.content)
    document = f
    rsrcmgr = PDFResourceManager()
    laparams = LAParams()
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    content = ''
    for page in PDFPage.get_pages(document):
        interpreter.process_page(page)
        layout = device.get_result()
        for element in layout:
            if isinstance(element, LTTextBoxHorizontal):
                content = content + element.get_text().replace('\n', " ")
    return content

In [3]:
main_url = r'https://citeseerx.ist.psu.edu/search?q=mobile+computing+and+applications&submit.x=0&submit.y=0&sort=rlv&t=doc'
page = BeautifulSoup(requests.get(main_url).text, 'html.parser')
article_urls = []
for _ in tqdm(range(21)):
    next_page_url = urljoin(main_url, page.find('div', id='pager').find('a').get('href'))
    for url in page.find_all('div', class_='result'):
        article_urls.append(urljoin(main_url, url.find('a').get('href')))
    page =  BeautifulSoup(requests.get(next_page_url).text, 'html.parser')

100%|██████████| 21/21 [01:04<00:00,  3.09s/it]


In [4]:
titles = []
authors = []
dates = []
abstracts = []
urls = []
keyWord = []

for i in tqdm(range(len(article_urls))):
    page = BeautifulSoup(requests.get(article_urls[i]).text, 'lxml')
    try:
        titles.append(page.find_all('meta')[4].get('content'))
    except:
        titles.append('null')
    try:
        authors.append(page.find_all('meta')[5].get('content'))
    except:
        authors.append('null')
    try:
        dates.append(page.find_all('meta')[6].get('content'))
    except:
        dates.append('null')
    try:
        abstracts.append(page.find('div', id='abstract').find('p').get_text())
    except:
        abstracts.append('null')
    try:
        urls.append(page.find('ul', id='clinks').find('a').get('href'))
    except:
        urls.append('null')
    key_words = []
    keys = page.find('div', id = 'keywords').find_all('a')
    for k in keys:
        key_words.append(k.get_text())
    keyWord.append(key_words)

kek = pd.DataFrame(
{
    'title': titles,
    'date': dates,
    'author':authors,
    'keywords': keyWord,
    'abstract': abstracts,
    'url': urls,
})

100%|██████████| 210/210 [08:47<00:00,  2.51s/it]


In [5]:
article_url = 'https://citeseerx.ist.psu.edu' + kek.url.values

In [6]:
texts = []
new_urls = []
for i in tqdm(article_url):
    try:
        texts.append(get_text(i))
        new_urls.append(i)
    except:
        texts.append('null')
        new_urls.append('null')
kek['text'] = texts
kek['url'] = new_urls

100%|██████████| 210/210 [56:26<00:00, 16.13s/it]   


In [7]:
kek.to_csv('adlet_extra.csv', index = False)